# 1章　ニューラルネットワークの復習

## 1.3 ニューラルネットワークの学習

ニューラルネットワークは, 最初に学習を行い, その学習されたパラメータを利用して推論を行う流れが一般的.  
推論は, 多クラス分類などの問題に答えを出す作業で, 学習は, 最適なパラメータを見つける作業.

### 1.3.1 損失関数

**損失** とは, 正解のデータとニューラルネットワークの出力結果を元に, どれだけ悪いかをスカラとして算出したもの.  
ニューラルネットワークの損失を求めるには **損失関数** を使用する.  
多クラスの分類の場合は, **交差エントロピー誤差** を用いる.

##### Softmax関数

$$
y_{k} = \frac{\exp(s_k)}{\sum_{i=1}^{n} \exp(s_i)}
$$

$k$ 番目の出力 $y_k$ を求める計算式を表している.  
$\text{Softmax}$ 関数の出力の各要素は $0.0$ より大きく $1.0$ 未満の実数になる.

##### 交差エントロピー誤差

$$
L = -\sum_{k} t_{k} \log{y_k} \qquad (式1.7)
$$

$t_{k}$ は $k$ 番目のクラスに対する教師ラベル

$$
L = -\frac{1}{N} \sum_{n} \sum_{k} t_{nk} \log{y_{nk}}
$$

これは上記の式1.7を $N$ 個分のデータ拡張しただけ.  
$1$ 個あたりの損失関数を求めることで, 統一した指標が得られる.
$\text{Softmax}$ 関数と交差エントロピー誤差を計算するレイヤを $\text{Sofftmax with Loss}$ レイヤとして実装する.

### 1.3.2 微分と勾配

ニューラルネットワークの学習の目標は, 損失をできるだけ少なくするパラメータを見つける.  
この時重要になるのが, 微分と勾配.  
$L$ をスカラ, $\mathbf{x}$ をベクトルとして, $L = f(\mathbf{x})$ という関数があるとする.  
この時の $\mathbf{x}$ の偏微分は,

$$
\frac{\partial L}{\partial \mathbf{x}}
=\left(\frac{\partial L}{\partial x_1}, \frac{\partial L}{\partial x_2}, \cdots , \frac{\partial L}{\partial x_n}\right)
$$

これを勾配という.  
$\mathbf{W}$ を $m\times n$ の行列とすると, $L = g(\mathbf{W})$ の勾配は, 

$$
\frac{\partial L}{\partial \mathbf{W}} = 
\begin{pmatrix}
\frac{\partial L}{\partial W_{11}} & \cdots & \frac{\partial L}{\partial W_{1n}} \\
\vdots & \ddots & \\
\frac{\partial L}{\partial W_{m1}} &  & \frac{\partial L}{\partial x_{mn}}
\end{pmatrix}
$$

### 1.3.3 チェインルール

**誤差逆伝播法**　を理解する上で, 大事なのは　**チェインルール(連鎖律)**　である.  
これは, 合成関数に関する微分の法則である.  

### 1.3.4.3 Repeatノード

In [1]:
import numpy as np

D, N = 8, 7
x = np.random.randn(1, D)
y = np.repeat(x, N, axis=0)
#print(x)
#print(y)

dy = np.random.randn(N, D)
dx = np.sum(dy, axis=0, keepdims=True)
#print(dy)
#print(dx)

#dx = np.sum(dy, axis=0, keepdims=False)
#print(dx)

`np.repeat()` は, 要素の複製を行う.  
`axis=0` は列に沿った操作, `axis=1` は行に沿った操作.  
`keepdims=True` は, 結果の配列が元の配列 `dy` と同じ数の次元をもつ.

### 1.3.4.4 Sumノード

In [2]:
x = np.random.randn(N, D)
y = np.sum(x, axis=0, keepdims=True)

dy = np.random.randn(1, D)
dx = np.repeat(x, N, axis=0)

Sumノードの順伝播は `np.sum()` メソッド, 逆伝播は, `np.repeat()`

### 1.3.4.5 MatMulノード

In [3]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

- `params`: 重みパラメータをリストで保持します。この例では、1つの重み行列 `W` が要素として入ている
- `grads`: 勾配(gradient)をリストで保持.  `W` に対応する勾配が要素として入る.
- `x`: 入力データ(forwardで受け取ったデータ)を保持するための変数.  

- `__init__(self, W)`: 引数 `W` として重み行列を受け取り, `params` と `grads` を初期化. 同時に, 入力データ保持用の `x` は `None` に初期化.

- `forward(self, x)`: 順伝播(forward propagation)処理を行うメソッド.
    - 最初に, `params` から重み行列 `W` を取り出す.
    - 入力データ `x` と重み行列 `W` の行列積を計算して、出力を `out` に代入.
    - 入力データ `x` を `self.x` に保持する.
    - 計算した出力を返す.

- `backward(self, dout)`: 逆伝播(backward propagation)処理を行うメソッド.
    - 最初に、`params` から重み行列 `W` を取り出す.
    - 出力に対する勾配 `dout` と、重み行列 `W` の転置行列との行列積を計算して、入力に対する勾配 `dx` を求める.
    - 入力データ `self.x` の転置行列と、出力に対する勾配 `dout` の行列積を計算して、重み行列 `W` の勾配 `dW` を求める.
    - `grads` の最初の要素 (重み行列に対応する勾配) を、計算した `dW` で更新.
    - 計算した入力に対する勾配 `dx` を返す.